# Structural Bioinformatics WS1516
## Global Sequence Alignment Implementation
**Author**: Charlie Hoyt

**Date**: 18 Jan 2016

This notebook outlines the implemention of the [Needleman-Wunsch](https://en.wikipedia.org/wiki/Needleman%E2%80%93Wunsch_algorithm) algorithm to perform a global alignment on sequence fragments from hemoglobin and myoglobin. It makes use of the [BLOSUM40](ftp://ftp.ncbi.nih.gov/blast/matrices/BLOSUM40) scoring matrix from the NCBI for sequence alignments.


### BLOSUM40 notes from the NCBI:
- Matrix made by matblas from blosum40.iij
- column uses minimum score
- BLOSUM Clustered Scoring Matrix in 1/4 Bit Units
- Blocks Database = /data/blocks_5.0/blocks.dat
- Cluster Percentage: >= 40
- Entropy =   0.2851, Expected =  -0.2090

In [1]:
blosum40_data = """
A  R  N  D  C  Q  E  G  H  I  L  K  M  F  P  S  T  W  Y  V  B  Z  X  *
A  5 -2 -1 -1 -2  0 -1  1 -2 -1 -2 -1 -1 -3 -2  1  0 -3 -2  0 -1 -1  0 -6 
R -2  9  0 -1 -3  2 -1 -3  0 -3 -2  3 -1 -2 -3 -1 -2 -2 -1 -2 -1  0 -1 -6 
N -1  0  8  2 -2  1 -1  0  1 -2 -3  0 -2 -3 -2  1  0 -4 -2 -3  4  0 -1 -6 
D -1 -1  2  9 -2 -1  2 -2  0 -4 -3  0 -3 -4 -2  0 -1 -5 -3 -3  6  1 -1 -6 
C -2 -3 -2 -2 16 -4 -2 -3 -4 -4 -2 -3 -3 -2 -5 -1 -1 -6 -4 -2 -2 -3 -2 -6 
Q  0  2  1 -1 -4  8  2 -2  0 -3 -2  1 -1 -4 -2  1 -1 -1 -1 -3  0  4 -1 -6 
E -1 -1 -1  2 -2  2  7 -3  0 -4 -2  1 -2 -3  0  0 -1 -2 -2 -3  1  5 -1 -6 
G  1 -3  0 -2 -3 -2 -3  8 -2 -4 -4 -2 -2 -3 -1  0 -2 -2 -3 -4 -1 -2 -1 -6 
H -2  0  1  0 -4  0  0 -2 13 -3 -2 -1  1 -2 -2 -1 -2 -5  2 -4  0  0 -1 -6 
I -1 -3 -2 -4 -4 -3 -4 -4 -3  6  2 -3  1  1 -2 -2 -1 -3  0  4 -3 -4 -1 -6 
L -2 -2 -3 -3 -2 -2 -2 -4 -2  2  6 -2  3  2 -4 -3 -1 -1  0  2 -3 -2 -1 -6 
K -1  3  0  0 -3  1  1 -2 -1 -3 -2  6 -1 -3 -1  0  0 -2 -1 -2  0  1 -1 -6 
M -1 -1 -2 -3 -3 -1 -2 -2  1  1  3 -1  7  0 -2 -2 -1 -2  1  1 -3 -2  0 -6 
F -3 -2 -3 -4 -2 -4 -3 -3 -2  1  2 -3  0  9 -4 -2 -1  1  4  0 -3 -4 -1 -6 
P -2 -3 -2 -2 -5 -2  0 -1 -2 -2 -4 -1 -2 -4 11 -1  0 -4 -3 -3 -2 -1 -2 -6 
S  1 -1  1  0 -1  1  0  0 -1 -2 -3  0 -2 -2 -1  5  2 -5 -2 -1  0  0  0 -6 
T  0 -2  0 -1 -1 -1 -1 -2 -2 -1 -1  0 -1 -1  0  2  6 -4 -1  1  0 -1  0 -6 
W -3 -2 -4 -5 -6 -1 -2 -2 -5 -3 -1 -2 -2  1 -4 -5 -4 19  3 -3 -4 -2 -2 -6 
Y -2 -1 -2 -3 -4 -1 -2 -3  2  0  0 -1  1  4 -3 -2 -1  3  9 -1 -3 -2 -1 -6 
V  0 -2 -3 -3 -2 -3 -3 -4 -4  4  2 -2  1  0 -3 -1  1 -3 -1  5 -3 -3 -1 -6 
B -1 -1  4  6 -2  0  1 -1  0 -3 -3  0 -3 -3 -2  0  0 -4 -3 -3  5  2 -1 -6 
Z -1  0  0  1 -3  4  5 -2  0 -4 -2  1 -2 -4 -1  0 -1 -2 -2 -3  2  5 -1 -6 
X  0 -1 -1 -1 -2 -1 -1 -1 -1 -1 -1 -1  0 -1 -2  0  0 -2 -1 -1 -1 -1 -1 -6 
* -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6 -6  1"""
blosum40_data = iter(blosum40_data.strip().split("\n"))
header = next(blosum40_data).split()
blosum40 = {}
for line in blosum40_data:
    row_index, *row_data = line.split()
    for column, row_datum in zip(header, row_data):
        blosum40[row_index, column] = int(row_datum)

In [15]:
import numpy as np

def align(a, b, s, gap=-2):    
    """Prints the alignment of string a and b"""
    
    """
    scores
    At position scores[i,j], this matrix stores the maximum score for
    choosing the best of matching, inserting, or deleting. This 
    allows for memoizing of smaller cases and alows this algorithm
    to make use of the dynamic programming paradigm
    """
    scores = np.zeros((len(a) + 1, len(b) + 1), dtype=int)
    
    """
    breadcrumbs
    At each step, the algorithm chooses the maximum of matching, 
    inserting, or deleting. The breadcrumbts stores for element (i,j) 
    the tuple representing the location of the maximum, at either:
    (i-1,j-1), (i-1,j), or (i,j-1)
    
    """
    breadcrumbs = {}
    
    # initialization
    for i in range(len(a) + 1):
        scores[i, 0] = gap * i
        breadcrumbs[i, 0] = (i - 1, 0)
    for j in range(len(b) + 1):
        scores[0, j] = gap * j
        breadcrumbs[0, j] = (0, j - 1)
    
    # calculation of optimal substructures
    for i, a_char in enumerate(a, start=1):
        for j, b_char in enumerate(b, start=1):
            # apply recurrence in order of match, insertion, deletion
            recurrences = {
                (i - 1, j - 1) : scores[i - 1, j - 1] + s[a_char, b_char],
                (i - 1, j)     : scores[i - 1, j] + gap, 
                (i, j - 1)     : scores[i, j - 1] + gap 
            }
            # store the position of the highest score for backtracking
            breadcrumbs[i, j] = max(recurrences, key=recurrences.get)
            # store the highest score for higher recurrences
            scores[i, j] = recurrences[breadcrumbs[i, j]]
    
    fmt = "Input\nA:{} {}\nB:{} {}\n\nScores\n{}\n\nBreadcrumbs"
    #print(fmt.format(a, len(a), b, len(b), scores))
    
    """
    Now that the results have been built, they need to be assembled.
    This makes use of the breadcrumbs by first looking at the final
    position (where the score is necessarily the highest), and backtracking.
    Because the algorithm is backtracking, it builds the aligned strings
    backwards and they are ultimately reversed before printing.
    
    Through each iteration, the current position (a_current, b_current) is
    compared to the previous position stored in the breadcrumbs at
    breadcrumbs[a_current, b_current]. The type of match is calculated, and
    the appropriate character is concatenated to the alignment strings,
    sa, and sb. 
    """
    # initialization
    a_aligned, b_aligned = "", ""
    a_current, b_current = len(a), len(b)
    
    # back-tracking
    while 0 != a_current or 0 != b_current:
        a_predecessor, b_predecessor = breadcrumbs[a_current, b_current]
        if a_current - 1 == a_predecessor and b_current - 1 == b_predecessor:
            #print("M ", end="") # matching
            a_aligned += a[a_predecessor]
            b_aligned += b[b_predecessor]
        elif a_current - 1 == a_predecessor and b_current == b_predecessor:
            #print("I ", end="") # insertion
            a_aligned += a[a_predecessor]
            b_aligned += "-"
        elif a_current == a_predecessor and b_current - 1 == b_predecessor:
            #print("D ", end="") # deletion
            a_aligned += "-"
            b_aligned += b[b_predecessor]
            
        #print("({},{}) => ({},{})".format(a_current, b_current, a_predecessor, b_predecessor))
        a_current, b_current = a_predecessor, b_predecessor
    
    a_aligned = a_aligned[::-1]
    b_aligned = b_aligned[::-1]
    #print("\nAlignment\n{}\n{}".format(a_aligned, b_aligned))
    return scores[len(a), len(b)]

In [16]:
s = {
    ('A','A'): 10,
    ('A','G'): -1,
    ('A','C'): -3,
    ('A','T'): -4,
    ('G','A'): -1,
    ('G','G'): 7,
    ('G','C'): -5,
    ('G','T'): -3,
    ('C','A'): -3,
    ('C','G'): -5,
    ('C','C'): 9,
    ('C','T'): 0,
    ('T','A'): -4,
    ('T','G'): -3,
    ('T','C'): 0,
    ('T','T'): 8
}
align("AGACTAGTTAC", "CGAGACGT", s, gap=-5)

16

In [17]:
hemoglobin = "KTEAEMKASEDLKKHGT"
myoglobin = "HGSAQVKGHG"
align(hemoglobin, myoglobin, blosum40, gap=-8)

-21

In [18]:
s = {
    ('A','A'): 5,
    ('A','R'): -2,
    ('A','N'): -1,
    ('A','D'): -2,
    ('R','A'): -2,
    ('R','R'): 7,
    ('R','N'): -1,
    ('R','D'): -2,
    ('N','A'): -1,
    ('N','R'): -1,
    ('N','N'): 7,
    ('N','D'): 2,
    ('D','A'): -2,
    ('D','R'): -2,
    ('D','N'): 2,
    ('D','D'): 8
}
align("DRDANND", "ARDNRND", s, gap=-10)

26

In [39]:
import itertools as itt

s = {
    ('A','A'): 5,
    ('A','R'): -2,
    ('A','N'): -1,
    ('A','D'): -2,
    ('R','A'): -2,
    ('R','R'): 7,
    ('R','N'): -1,
    ('R','D'): -2,
    ('N','A'): -1,
    ('N','R'): -1,
    ('N','N'): 7,
    ('N','D'): 2,
    ('D','A'): -2,
    ('D','R'): -2,
    ('D','N'): 2,
    ('D','D'): 8
}

alphabet = "ARND"
target ="RRDNARD"

substring_len = 4
threshold = 3
valid_count = 0

# iterate over all possible elements of alphabet^substring_len
for perm in itt.product(alphabet, repeat=substring_len):
    # start with worst possible score
    max_score = min(s.values()) * substring_len
    
    # slide window and calculate score for each
    for i in range(len(target) - substring_len):
        score = sum(s[matching] for matching in zip(perm, target[i:i + substring_len]))
        if max_score < score:
            max_score = score

    if threshold <= max_score:
        print(''.join(perm), max_score)
        valid_count += 1
    
print(valid_count)

AAAR 9
AANA 8
AANR 3
AANN 5
AADA 3
AADR 3
AADN 11
AADD 6
ARAR 9
ARAN 10
ARAD 5
ARRN 10
ARRD 5
ARNA 8
ARNR 6
ARNN 14
ARND 9
ARDA 12
ARDR 12
ARDN 20
ARDD 15
ANAA 8
ANAR 17
ANAN 9
ANAD 8
ANRA 4
ANRR 10
ANNA 12
ANNR 11
ANNN 6
ANND 5
ANDA 7
ANDR 10
ANDN 12
ANDD 7
ADAA 10
ADAR 12
ADAN 4
ADAD 3
ADRA 10
ADRR 5
ADRN 4
ADRD 3
ADNA 18
ADNR 11
ADNN 12
ADND 11
ADDA 13
ADDR 6
ADDN 11
ADDD 6
RAAA 9
RAAR 9
RAAN 10
RAAD 5
RARA 9
RARN 10
RARD 5
RANA 17
RANR 10
RANN 14
RAND 10
RADA 12
RADR 12
RADN 20
RADD 15
RRAA 11
RRAR 11
RRAN 19
RRAD 14
RRRA 11
RRRR 11
RRRN 19
RRRD 14
RRNA 17
RRNR 15
RRNN 23
RRND 18
RRDA 21
RRDR 21
RRDN 29
RRDD 24
RNAA 13
RNAR 17
RNAN 11
RNAD 8
RNRA 13
RNRR 10
RNRN 11
RNRD 6
RNNA 21
RNNR 14
RNNN 15
RNND 14
RNDA 16
RNDR 13
RNDN 21
RNDD 16
RDAA 19
RDAR 12
RDAN 13
RDAD 12
RDRA 19
RDRR 12
RDRN 13
RDRD 12
RDNA 27
RDNR 20
RDNN 21
RDND 20
RDDA 22
RDDR 15
RDDN 20
RDDD 15
NAAA 4
NAAR 13
NAAN 5
NAAD 4
NARR 6
NANA 9
NANR 7
NANN 6
NADA 4
NADR 6
NADN 12
NADD 7
NRAA 4
NRAR 13
NRAN 11
NRAD 6
NRRA 3
